In [1]:
import tequila as tq
import numpy as np
from tequila.hamiltonian import QubitHamiltonian, paulis
from tequila.grouping.binary_rep import BinaryHamiltonian

The following examples shows how to partition a given Hamiltonian into commuting parts and how to find the unitary transformation needed to transform the commuting terms into qubit-wise commuting form that is easy to measure. 

The Hamiltonian is simply 
$$ H = \sigma_z(0)\sigma_z(1) + \sigma_y(0)\sigma_y(1) + \sigma_x(0)\sigma_x(1) + \sigma_x(0)$$
where $\sigma_z(0)\sigma_z(1)$, $\sigma_y(0)\sigma_y(1)$ does not commute with $\sigma_x(0)$, so two separate measurements are needed.

In [2]:
H = paulis.Z(0) * paulis.Z(1) + paulis.Y(0) * paulis.Y(1) + \
    paulis.X(0) * paulis.X(1) + paulis.X(0) 

Here we use the binary representation of the Hamiltonian for partitioning. The method commuting_groups gets back a list of BinaryHamiltonian whose terms are mutually commuting. 

Call to_qubit_hamiltonian to visualize.

In [3]:
binary_H = BinaryHamiltonian.init_from_qubit_hamiltonian(H)
commuting_parts = binary_H.commuting_groups()

In [4]:
print(len(commuting_parts)) # Number of measurements needed
print(commuting_parts[0].to_qubit_hamiltonian())
print(commuting_parts[1].to_qubit_hamiltonian())

2
+0.0000i+1.0000X(0)X(1)+1.0000X(0)
+0.0000i+1.0000Z(0)Z(1)+1.0000Y(0)Y(1)


The second group of terms $H_2$ are not currently qubit-wise commuting and cannot be directly measured on current hardware. They require further unitary transformation $U$ to become qubit-wise commuting. The following code identifies two bases (list of BinaryPauliString) that encodes the unitary transformation as
$$ U = \prod_i \frac{1}{\sqrt{2}} (\text{old_basis}[i] + \text{new_basis}[i])$$
such that $UH_2U$ is qubit-wise commuting.

In [186]:
qubit_wise_parts, old_basis, new_basis = commuting_parts[1].get_qubit_wise()

In [187]:
def display_basis(basis):
    for term in basis:
        print(QubitHamiltonian.init_from_paulistring(term.to_pauli_strings()))
print('Old Basis')
display_basis(old_basis)
print('\nNew Basis')
display_basis(new_basis)

Old Basis
+1.0000Y(0)Y(1)
+1.0000X(0)X(1)

New Basis
+1.0000X(0)
+1.0000Y(1)


The transfromed term $UH_2U$ is qubit-wise commuting. 

In [7]:
print(qubit_wise_parts.to_qubit_hamiltonian())

+0.0000i-1.0000X(0)Y(1)+1.0000X(0)


Trying to construct the circuit for the unitary transformation to implement the measurement scheme

In [199]:
# proposition of how the code could work
# would move that then into the tequila objectives to transform all expectationvalues at once
# the first term which was already qwc is now still transformed (by single qubit rotations), can it be detected and skipped in the code below?

def optimize_measurements(H, U):
    binary_H = BinaryHamiltonian.init_from_qubit_hamiltonian(H)
    commuting_parts = binary_H.commuting_groups()
    result = None
    for cH in commuting_parts:
        qwc, bas1, bas2 = cH.get_qubit_wise()
        Um = tq.QCircuit()
        for i in range(len(bas1)):
            sigma = bas1[i].to_pauli_strings()
            tau = bas2[i].to_pauli_strings()
            V0=tq.gates.ExpPauli(angle=-tq.numpy.pi/2, paulistring=sigma) 
            V1=tq.gates.ExpPauli(angle=-tq.numpy.pi/2, paulistring=tau)
            V2=tq.gates.ExpPauli(angle=-tq.numpy.pi/2, paulistring=sigma)
            Um += V0 + V1 + V2
        Etmp = tq.ExpectationValue(H=qwc.to_qubit_hamiltonian(), U=U+Um)
        if result is None: # currently necessary
            result = Etmp
        else:
            result += Etmp
    
    return result
        
    
    



In [201]:
# test it out

E1 = tq.ExpectationValue(H=H, U=U)
E2 = optimize_measurements(H, U)

print(tq.simulate(E1, variables))
print(tq.simulate(E2, variables))

1.8414709568023682
1.8414710760116577
